# SVM LAB

The Support Vector Classifier lab discussed how support vector machines are used to classify data using linear boundaries.  
The dependent variable in IRIS data is scattered every where, 
making it hard to create a linear boundary for classifying the observations. 
Below we will fit a SVM model with a polynomial kernal to classify the observations into classes 
 * Setosa, 
 * Virginica and 
 * Versicolor.

### Load R Library with Iris Data

In [ ]:
library("e1071")

Look at the data a little

In [ ]:
head(iris,5)

Attach the iris data in memory so you can reference it directly.

In [ ]:
attach(iris)

Now create a model using the "svm" function using a linear kernal first and evaluate its performance.

For now we will limit ourself to the using just two features:
 * Sepal.Length
 * Sepal.Width

In [ ]:
library(caTools)
set.seed(1000) # set.seed() will help us to reproduce the results.
split = sample.split(iris$Species, SplitRatio=0.66)

train_data  = subset(iris, split==TRUE)

# Test data will have the rest 30% of data
test_data  = subset(iris, split==FALSE)

In [ ]:
# Fit the model using Sepal.Length and Sepal.Width as the predictors. Use a linear kernal to fit the model.
svm.model <- svm(Species ~ Sepal.Length + Sepal.Width, data = train_data, kernel = "linear")

# Plot the Species and show the support vectors on graph. 
# the + signs are support vectors
plot(iris$Sepal.Length, iris$Sepal.Width, col = as.integer(iris[, 5]), # color the points based on species 
     pch = c("o","+")[1:150 %in% svm.model$index + 1], 
       # Mark the support vectors with a `+` sign and test with a `o` sign
       # "1:150 %in% svm.model$index" will generate a vector of size 150
       # with TRUE and FALSE values. A TRUE is assigned if the value is a 
       # support vector. Addimg one to the vector will give values 1 and 2 
       # instead of TRUE(1) and FALSE(0). Every 1 in the vector is displayed
       # as o and 2 is displayed as +. 
     cex = 2, 
     xlab = "Sepal length", ylab = "Sepal width")

In [ ]:
# Plot the Species by splitting the feature space into three different regions according to species class
plot(svm.model, iris, Sepal.Width ~ Sepal.Length, # Plot the model predictions with sepal.width on y-axis and sepal.length
                                                  # on x-axis
     slice = list(sepal.width = 1, sepal.length = 2)) # a list of named numeric values for the dimensions held constant 
                                                      # slice is needed if more than two variables are used.

In [ ]:
# Make predictions of species using the svm model built
svm.pred  <- predict(svm.model, test_data[,-5]) 

# Build a confusion matrix for the predictions made against the original classes of flowers
library(caret)
confusionMatrix(svm.pred, test_data[,5])

The svm model did not do a great job with a linear kernal. The accuracy of the model is 74.5
         
         (17+10+11)/51  --- number of TRUE predictions/total observations 
         
**Reference: ** [Confusion matrix function and its results](http://rpubs.com/prcuny/161764)

#### Fit the model using a Non-Linear Kernal. 

In [ ]:
# Fit the model using a polynomial kernal and Sepal.Length, Sepal.Width as predictor variables.
svm.model <- svm(Species ~ Sepal.Length + Sepal.Width, data = train_data, kernel = 'polynomial', degree=8, coef0=1)
                      # For polynomial kernels we use the parameter degree to adjust the polynomial order. 
                      # For radial kernels we use the gamma parameter to adjust the y value.
                      # Independent term in kernel function. It is only significant in ‘polynomial’ 
                      # and ‘sigmoid’ kernals
                  
plot(svm.model, iris, Sepal.Width ~ Sepal.Length,      # Plot the predictions
     slice = list(Sepal.Width = 1, Sepal.Length = 2)) 

In [ ]:
svm.pred  <- predict(svm.model, test_data[,-5]) 
confusionMatrix(svm.pred, test_data[,5]) # show the confusion matrix

There is no improvement in the accuracy of the model even after using a polynomial of degree 8.  
We previously only used just two attributes for making predictions. 
We will now use all independent variables for building the model.

In [ ]:
svm.model <- svm(Species ~ ., data = train_data, kernel = 'polynomial', degree=8, coef0=1)

In [ ]:
svm.pred  <- predict(svm.model, test_data[,-5]) 
confusionMatrix(svm.pred, test_data[,5]) # show the confusion matrix

Using all variables as predictors we got over 90% accuracy in our model. 

#### Radial Kernal

Now we will investigate using a radial kernel!

In [ ]:
svm.model <- svm(Species ~., data=train_data, kernel='radial', gamma=1, cost=1)

In [ ]:
yhat <- predict(svm.model, test_data)
confusionMatrix(yhat, test_data[,'Species'])

#### Tune the model

Tuning SVM to find the best cost and gamma..

In [ ]:
svm_tune <- tune(svm, train.x=train_data[,-5], train.y=train_data[,5], 
              kernel="radial", ranges=list(cost=10^(-1:2), gamma=c(.5,1,2)))

print(svm_tune)

After you find the best cost and gamma, you can create AVM model again and run the model

In [ ]:
svm_model_after_tune <- svm(Species ~ ., data=train_data, kernel="radial", cost=1, gamma=0.5)
summary(svm_model_after_tune)

In [ ]:
yhat <- predict(svm_model_after_tune, test_data)
confusionMatrix(yhat, test_data[,'Species'])

We have now gotten 100% accurate results after tuning the model. 

**Additional Reference:** 
  * [SVM tutorial 1](http://www.di.fc.ul.pt/~jpn/r/svm/svm.html#non-linearly-separable-data) 
  * [SVM tutorial 2](https://rpubs.com/ryankelly/svm)
  
# Save your Notebooks